<a href="https://colab.research.google.com/github/Nivedithabp/LLM_Neo4j_llama-3.1_nvidia/blob/main/llm/nvidia_neo4j_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet langchain-nvidia-ai-endpoints langchain-community neo4j langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [3]:
import os
from typing import Any, Dict, List, Optional, Tuple, Type

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import \
    format_to_openai_function_messages
from langchain.agents.output_parsers.openai_tools import \
    OpenAIToolsAgentOutputParser
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun,
                                         CallbackManagerForToolRun)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import AIMessage, HumanMessage
from langchain.tools import BaseTool
from langchain.tools.render import format_tool_to_openai_function

In [18]:
os.environ["NEO4J_URI"] = "bolt://3.228.0.76:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "concern-ease-compiler"

graph = Neo4jGraph(refresh_schema=False)

In [19]:
os.environ["NVIDIA_API_KEY"] = "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"
llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")

In [20]:
result = llm.invoke("How to use LLMs in combination with Graph Databases? Be concise!")
print(result.content)

Using Large Language Models (LLMs) with graph databases can be a powerful combination for knowledge graph-based applications, such as question answering, recommender systems, and knowledge discovery. Here's a concise overview of how to combine LLMs with graph databases:

**Architecture:**

1. **Knowledge Graph Construction**: Build a graph database (e.g., Neo4j, Amazon Neptune) by integrating data from various sources.
2. **LLM Integration**: Use an LLM (e.g., BERT, RoBERTa) to generate node embeddings, which capture the semantic meaning of nodes in the graph.
3. **Query Processing**: When a query is issued, use the LLM to generate a query embedding.
4. **Graph Search**: Use the query embedding to search the graph database, retrieving relevant nodes and their relationships.

**Example Use Cases:**

1. **Question Answering**: Use an LLM to generate a query embedding for a natural language question, and then search the graph database for answers.
2. **Recommender Systems**: Use an LLM to

The schema centers on the Case node, which links various aspects of a drug safety report, including the drugs involved, reactions experienced, outcomes, and therapies prescribed. Each drug is characterized by whether it is primary, secondary, concomitant, or interacting. Cases are also associated with information about the manufacturer, the age group of the patient, and the source of the report.

In [21]:
graph.query(
    "CREATE FULLTEXT INDEX drug IF NOT EXISTS FOR (d:Drug) ON EACH [d.name];"
)
graph.query(
    "CREATE FULLTEXT INDEX manufacturer IF NOT EXISTS FOR (d:Manufacturer) ON EACH [d.manufacturerName];"
)

[]

In [22]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2) to each word, then combines them using the AND
    operator. Useful for mapping movies and people from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


candidate_query = """
CALL db.index.fulltext.queryNodes($index, $fulltextQuery, {limit: $limit})
YIELD node
RETURN coalesce(node.manufacturerName, node.name) AS candidate,
       labels(node)[0] AS label
"""


def get_candidates(input: str, type: str, limit: int = 3) -> List[Dict[str, str]]:
    """
    Retrieve a list of candidate entities from database based on the input string.

    This function queries the Neo4j database using a full-text search. It takes the
    input string, generates a full-text query, and executes this query against the
    specified index in the database. The function returns a list of candidates
    matching the query, with each candidate being a dictionary containing their name
    (or title) and label (either 'Person' or 'Movie').
    """
    ft_query = generate_full_text_query(input)
    candidates = graph.query(
        candidate_query, {"fulltextQuery": ft_query, "index": type, "limit": limit}
    )
    return candidates

In [23]:
@tool
def get_side_effects(
    drug: Optional[str] = Field(
        description="disease mentioned in the question. Return None if no mentioned."
    ),
    min_age: Optional[int] = Field(
        description="Minimum age of the patient. Return None if no mentioned."
    ),
    max_age: Optional[int] = Field(
        description="Maximum age of the patient. Return None if no mentioned."
    ),
    manufacturer: Optional[str] = Field(
        description="manufacturer of the drug. Return None if no mentioned."
    ),
):
    """Useful for when you need to find common side effects."""
    params = {}
    filters = []
    side_effects_base_query = """
    MATCH (c:Case)-[:HAS_REACTION]->(r:Reaction), (c)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
    """
    if drug and isinstance(drug, str):
        candidate_drugs = [el["candidate"] for el in get_candidates(drug, "drug")]
        if not candidate_drugs:
            return "The mentioned drug was not found"
        filters.append("d.name IN $drugs")
        params["drugs"] = candidate_drugs

    if min_age and isinstance(min_age, int):
        filters.append("c.age > $min_age ")
        params["min_age"] = min_age
    if max_age and isinstance(max_age, int):
        filters.append("c.age < $max_age ")
        params["max_age"] = max_age
    if manufacturer and isinstance(manufacturer, str):
        candidate_manufacturers = [
            el["candidate"] for el in get_candidates(manufacturer, "manufacturer")
        ]
        if not candidate_manufacturers:
            return "The mentioned manufacturer was not found"
        filters.append(
            "EXISTS {(c)<-[:REGISTERED]-(:Manufacturer {manufacturerName: $manufacturer})}"
        )
        params["manufacturer"] = candidate_manufacturers[0]

    if filters:
        side_effects_base_query += " WHERE "
        side_effects_base_query += " AND ".join(filters)
    side_effects_base_query += """
    RETURN d.name AS drug, r.description AS side_effect, count(*) AS count
    ORDER BY count DESC
    LIMIT 10
    """
    print(f"Using parameters: {params}")
    data = graph.query(side_effects_base_query, params=params)
    return data

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that finds information about common side effects. "
            "If tools require follow up questions, "
            "make sure to ask the user for clarification. Make sure to include any "
            "available options that need to be clarified in the follow up questions "
            "Do only the things the user specifically requested. ",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [25]:
tools = [get_side_effects]
llm_with_tools = llm.bind_tools(tools=tools)

agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"])
        if x.get("chat_history")
        else [],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


class Output(BaseModel):
    output: Any


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput, output_type=Output
)

In [26]:
agent_executor.invoke(
    {
        "input": "What are the most common side effects when using lyrica for people below 35 years old?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `get_side_effects` with `{'drug': 'lyrica', 'max_age': 35, 'manufacturer': 'Pfizer', 'min_age': 18}`


Using parameters: {'drugs': ['LYRICA', 'LYRICA CR'], 'min_age': 18, 'max_age': 35, 'manufacturer': 'PFIZER'}
[{'drug': 'LYRICA', 'side_effect': 'Sinusitis', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Hypoacusis', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Fall', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Brain injury', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Amnesia', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Off label use', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Impaired quality of life', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Somnolence', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Drug ineffective for unapproved indication', 'count': 1}]The most common side effects of using Lyrica for people below 35 years old are sinusitis, hypoacusis, fall, brain injury, amnesia, off-label use, i

{'input': 'What are the most common side effects when using lyrica for people below 35 years old?',
 'output': 'The most common side effects of using Lyrica for people below 35 years old are sinusitis, hypoacusis, fall, brain injury, amnesia, off-label use, impaired quality of life, somnolence, and drug ineffective for unapproved indication.'}

In [27]:
agent_executor.invoke(
    {
        "input": "What are the most common side effects when for drugs manufactured by acadia?"
    }
)




> Entering new AgentExecutor chain...

Invoking: `get_side_effects` with `{'drug': 'None', 'min_age': 18, 'max_age': 65, 'manufacturer': 'acadia'}`


Using parameters: {'drugs': ['Carboplatin (Non GSK Comparator)', "One a day women's", 'CHLORASEPTIC SORE THROAT'], 'min_age': 18, 'max_age': 65, 'manufacturer': 'ACADIA PHARMACEUTICALS'}
[]I don't have enough information to answer your question. I will need more information to look up which common side effects were reported for drugs that Acadia manufactured. Can you provide the name of the drug or its intended use or anything else that might help me narrow down the results?

> Finished chain.


{'input': 'What are the most common side effects when for drugs manufactured by acadia?',
 'output': "I don't have enough information to answer your question. I will need more information to look up which common side effects were reported for drugs that Acadia manufactured. Can you provide the name of the drug or its intended use or anything else that might help me narrow down the results?"}